In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',500)

## Getting State geography

In [2]:
url = 'https://www.census.gov/geographies/reference-files/2010/geo/state-area.html'
geo = pd.read_html(url)[0]
geo = geo.iloc[:, [0,3]]
geo.columns = ['state', 'land_area']
geo = geo.iloc[3:,:]


## Population Data

In [3]:
sex_dict = {0:'Total', 1: 'Male', 2:'Female'}
race_dict = {1: 'White', 2: 'Black', 3: 'Other', 4: 'Other', 5: 'Other', 6: 'Other'}

df = pd.read_csv('../data/raw/sc-est2019-alldata6.csv', usecols=['REGION', 'NAME','ORIGIN', 'RACE', 
        'AGE', 'SEX', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016',
       'POPESTIMATE2017', 'POPESTIMATE2018', 'POPESTIMATE2019'])

df = df.drop(df[df['AGE'] <18].index)
df['SEX_map'] = df['SEX'].map(sex_dict)
df['RACE_map'] = df['RACE'].map(race_dict)
df['age_map'] = df['AGE'].apply(lambda x : 'minor' if x < 18 else ('adult' if x < 60 else 'elder'))

df = df.drop(['SEX', 'RACE', 'AGE'], axis=1)

df.columns = ['region', 'state', 'hispanic',  '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
              '2018', '2019', 'sex', 'race', 'age',]

## Age Fractions

In [4]:
df_age = df[['state', 'age', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019']]
df_older = df_age[df_age['age'] == 'elder']
df_older = df_older.groupby(['state']).sum().reset_index()
df_total = df_age.groupby(['state']).sum().reset_index()
df_age = df_older.merge(df_total, on = 'state')

df_age.columns = df_age.columns.str.replace('x', 'older')
df_age.columns = df_age.columns.str.replace('y', 'voting')

In [5]:
df_age.tail()

,state,2010_older,2011_older,2012_older,2013_older,2014_older,2015_older,2016_older,2017_older,2018_older,2019_older,2010_voting,2011_voting,2012_voting,2013_voting,2014_voting,2015_voting,2016_voting,2017_voting,2018_voting,2019_voting
46,Virginia,5719880,5904660,6097436,6291560,6500720,6711004,6923576,7140364,7351708,7555800,24675332,24976056,25292652,25554724,25785204,25978996,26164312,26372004,26541088,26698684
47,Washington,4881436,5066240,5256428,5453272,5664368,5885116,6104988,6320240,6526192,6723612,20640128,20961520,21234308,21482532,21811504,22197740,22647672,23089784,23462980,23807328
48,West Virginia,1699632,1735496,1772940,1807220,1842228,1873904,1902868,1933440,1962668,1986840,5867988,5882948,5889928,5886324,5875176,5855740,5827236,5790852,5759420,5730320
49,Wisconsin,4391764,4514448,4649336,4782176,4925792,5063904,5205112,5356072,5512192,5672084,17413956,17515844,17612896,17711748,17797808,17863340,17937276,18027924,18129464,18223348
50,Wyoming,414192,429356,446104,463372,478368,495068,511120,527972,543400,558660,1716004,1727720,1759676,1777800,1776652,1784256,1781900,1770900,1772248,1780100


## Race Distribution

In [6]:
df_race = df[['state', 'race', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019']]

df_race = df_race.groupby(['state', 'race']).sum()
df_race = df_race.groupby(level=0).apply(lambda x : x / x.sum()).reset_index()


In [7]:
df_race.head()

,state,race,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,Black,0.250305,0.251886,0.253690,0.255592,0.256880,0.258384,0.259248,0.259943,0.260451,0.260598
1,Alabama,Other,0.028674,0.029261,0.030218,0.030892,0.031552,0.032310,0.033128,0.033874,0.034552,0.035250
2,Alabama,White,0.721022,0.718853,0.716092,0.713515,0.711567,0.709306,0.707624,0.706184,0.704997,0.704152
3,Alaska,Black,0.033926,0.034606,0.036213,0.036802,0.036696,0.036751,0.037149,0.037470,0.037660,0.037506
4,Alaska,Other,0.247688,0.249927,0.251384,0.254314,0.257665,0.261016,0.264612,0.268850,0.272173,0.274928


## Gender Distribution

In [8]:
df_sex = df[['state', 'sex', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019']]
df_sex = df_sex[df_sex['sex'] != 'Total']
df_sex = df_sex.groupby(['state', 'sex']).sum()
df_sex = df_sex.groupby(level=0).apply(lambda x : x / x.sum()).reset_index()
df_sex = df_sex[df_sex['sex'] == 'Male'].drop('sex', axis=1)

In [9]:
df_sex.head()

,state,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Alabama,0.477551,0.477385,0.477446,0.477592,0.477409,0.477343,0.477089,0.476544,0.476144,0.475850
3,Alaska,0.522877,0.522686,0.524002,0.526158,0.526725,0.526832,0.526427,0.525221,0.524458,0.524034
5,Arizona,0.492280,0.492796,0.492929,0.492863,0.493119,0.493212,0.493206,0.493135,0.493200,0.493184
7,Arkansas,0.484409,0.484714,0.484773,0.484717,0.484646,0.484816,0.484896,0.484909,0.484647,0.484534
9,California,0.492095,0.491972,0.492114,0.492110,0.492288,0.492501,0.492631,0.492875,0.493052,0.493151


## Education

In [10]:
df = pd.read_csv('../data/raw/education_attainment.csv')

## Income

In [11]:
income = pd.read_csv('../data/raw/income.csv')

In [12]:
income.head()

,State,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000
0,Alabama,49936,51113,47221,44509,42278,47320,43464,42590,40933,39980,44476,42212,37952,37150,36629,37255,37603,35160,35424
1,Alaska,68734,72231,75723,75112,67629,72472,63648,57431,57848,61604,63989,62993,56418,55891,55063,51837,52774,57363,52847
2,Arizona,62283,61125,57100,52248,49254,52611,47044,48621,46896,45739,46914,47215,46657,45245,43846,41166,39734,42704,39783
3,Arkansas,49781,48829,45907,42798,44922,39376,39018,41302,38587,36538,39586,40795,37057,36658,34984,32002,32387,33339,29697
4,California,70489,69759,66637,63636,60487,60794,57020,53367,54283,56134,57014,55734,55319,51755,49222,49300,47437,47262,46816


## Gallup Party affiliation

In [13]:
party_affil = pd.read_csv('../data/raw/gallup.csv', usecols=['State', 'Democrat', 'Republican', 'Democratic advantage'])
party_affil.columns = ['state', 'dem', 'rep', 'dem_adv']
party_affil['no_party'] = 100-party_affil.dem-party_affil.rep

In [14]:
party_affil.head()

,state,dem,rep,dem_adv,no_party
0,Alabama,35,50,-15,15
1,Alaska,31,52,-21,17
2,Arizona,40,42,-2,18
3,Arkansas,36,45,-9,19
4,California,51,30,21,19


## 2020 data

In [15]:
age2020 = df_age[['state', '2019_older', '2019_voting']].rename(columns=
                                                                {'2019_older': 'older_pop', '2019_voting':'vote_pop'})

pop_density = age2020[['state', 'vote_pop']].merge(geo, on = 'state')
pop_density['pop_density'] = pop_density['vote_pop'] / pop_density['land_area']
education2020 = pd.read_csv('../data/raw/education_attainment.csv')
income2020 = income[['State','2018']].rename(columns={'State' : 'state', '2018': 'median_income'})


race2020 = df_race.pivot_table(index = 'state', columns = 'race', values = '2019').reset_index()
gender_2020 = df_sex[['state', '2019']].rename(columns={'2019': 'per_male'})

demo2020 = age2020.merge(gender_2020, on = 'state').merge(race2020, on = 'state').merge(
    pop_density[['state', 'pop_density']], on='state').merge(education2020, on='state').merge(
    income2020, on='state').merge(party_affil[['state', 'dem_adv', 'no_party']], on='state')
demo2020.set_index('state', inplace=True)

In [16]:
demo2020.head(10)

,older_pop,vote_pop,per_male,Black,Other,White,pop_density,educ_hs,educ_college,median_income,dem_adv,no_party
state,,,,,,,,,,,,
Alabama,4684736,15259516,0.475850,0.260598,0.035250,0.704152,301.303505,86.6,25.5,49936,-15,15
Alaska,546820,2206248,0.524034,0.037506,0.274928,0.687565,3.866263,93.3,30.2,68734,-21,17
Arizona,6970784,22553924,0.493184,0.049227,0.109960,0.840813,198.548550,87.5,29.7,62283,-2,18
Arkansas,2847420,9270596,0.484534,0.149378,0.045126,0.805496,178.160776,87.2,23.3,49781,-9,19
California,32463892,122470328,0.493151,0.065239,0.215547,0.719214,786.179960,83.8,34.2,70489,21,19
Colorado,4761792,17996868,0.501802,0.044079,0.076038,0.879883,173.644546,91.9,41.7,73034,9,17
Connecticut,3502928,11351388,0.482114,0.116292,0.072308,0.811400,2344.359356,90.9,39.6,72812,19,17
Delaware,1029284,3080768,0.477404,0.220548,0.066276,0.713177,1580.691637,89.8,31.3,65012,12,22
District of Columbia,482364,2310324,0.467096,0.438984,0.078166,0.482850,37874.163934,92.1,60.4,85750,59,19


In [17]:

from scipy import cluster as sp_cluster
from sklearn import cluster, neighbors

In [18]:
clean_data = sp_cluster.vq.whiten(demo2020.values)

In [19]:
k_means = cluster.KMeans(n_clusters=8, n_init=50)
k_means.fit(clean_data)
values = k_means.cluster_centers_.squeeze()
labels = k_means.labels_

In [20]:
clusters = sp_cluster.vq.kmeans(clean_data, 8)[0]

In [21]:
def choose_group(data, clusters):
    """
    Return the index of the cluster to which the rows in data
    are "closest" (in the sense of the L2-norm)
    """
    data = data[:,None] # add an axis for broadcasting
    distances = data - clusters
    groups = []
    for row in distances:
        dists = map(np.linalg.norm, row)
        groups.append(np.argmin(dists))
    return groups

In [22]:
groups = choose_group(clean_data, clusters)

In [23]:
groups = [np.argmin(map(np.linalg.norm, (clean_data[:,None] - clusters)[i])) for i in range(51)]


demo2020["kmeans_labels"] = labels

for _, group in demo2020.groupby("kmeans_labels"):
    group = group.index
    group.values.sort()
    print (group.values)


['Arizona' 'Arkansas' 'Indiana' 'Kentucky' 'Michigan' 'Missouri' 'Nevada'
 'New Mexico' 'Ohio' 'Oklahoma' 'Pennsylvania' 'Tennessee' 'West Virginia']
['District of Columbia']
['California' 'Florida' 'New York' 'Texas']
['Connecticut' 'Delaware' 'Illinois' 'Maryland' 'Massachusetts'
 'New Jersey' 'Rhode Island' 'Virginia']
['Alabama' 'Georgia' 'Louisiana' 'Mississippi' 'North Carolina'
 'South Carolina']
['Alaska' 'Idaho' 'Montana' 'Nebraska' 'North Dakota' 'South Dakota'
 'Utah' 'Wyoming']
['Hawaii']
['Colorado' 'Iowa' 'Kansas' 'Maine' 'Minnesota' 'New Hampshire' 'Oregon'
 'Vermont' 'Washington' 'Wisconsin']


In [24]:
demo2020.head()

,older_pop,vote_pop,per_male,Black,Other,White,pop_density,educ_hs,educ_college,median_income,dem_adv,no_party,kmeans_labels
state,,,,,,,,,,,,,
Alabama,4684736,15259516,0.475850,0.260598,0.035250,0.704152,301.303505,86.6,25.5,49936,-15,15,4
Alaska,546820,2206248,0.524034,0.037506,0.274928,0.687565,3.866263,93.3,30.2,68734,-21,17,5
Arizona,6970784,22553924,0.493184,0.049227,0.109960,0.840813,198.548550,87.5,29.7,62283,-2,18,0
Arkansas,2847420,9270596,0.484534,0.149378,0.045126,0.805496,178.160776,87.2,23.3,49781,-9,19,0
California,32463892,122470328,0.493151,0.065239,0.215547,0.719214,786.179960,83.8,34.2,70489,21,19,2


In [25]:
demo2020 = demo2020.reset_index()


In [26]:
demo2020.to_csv('../data/processed/demographics2020.csv', index=False)